In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
'''Boilerplate code'''
data = pd.read_csv('Churn_Modelling.csv')

# Preprocessing data

# Drop irrelavant features
data = data.drop(["RowNumber", "CustomerId", "Surname"], axis=1)

# Label Encoding on the gender Column
gender_le = LabelEncoder()
data['Gender'] = gender_le.fit_transform(data['Gender'])

# One-hot encode the 'Geography' column
geography_ohe = OneHotEncoder(sparse_output=False) # sparse_output=False is easier with pandas
geo_encoded_array = geography_ohe.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded_array, columns=geography_ohe.get_feature_names_out(['Geography']))

# Combine the new columns and drop the original 'Geography' column
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

# Divide the dataset into independent and dependent features

# since we are predicting customer churn, we'll be keeping exited as the dependent feature and everything else as the independent features
X = data.drop('Exited', axis=1)
y = data['Exited']

# Split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features [0-1]
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save the encoders and scaler for later use
with open('gender_le.pkl', 'wb') as file:
    pickle.dump(gender_le, file)

with open('geography_ohe.pkl', 'wb') as file:
    pickle.dump(geography_ohe, file)

# save the scaler object
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


In [3]:
## Define a function to create the model and try different parameters (KerasClassifier)

def create_model(neurons=32, layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))
    
    for _ in range(layers -1): 
        model.add(Dense(neurons, activation='relu'))

    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [4]:
# Create a keras classifer
model = KerasClassifier(neurons=16, layers=1,build_fn=create_model, epochs=50, batch_size=10, verbose=1)

In [ ]:
# Define the Grid Search parameters
param_grid = {
    'neurons': [16, 32, 64, 128], 
    'layers': [1,2],
    'epochs':[50, 100]
}

In [6]:
# Perform grid search 
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))


/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argu

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7919 - loss: 0.4695
Epoch 2/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8009 - loss: 0.4604
Epoch 2/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7932 - loss: 0.4636
Epoch 2/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7830 - loss: 0.4739
Epoch 2/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7737 - loss: 0.4983
Epoch 2/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7654 - loss: 0.5198
Epoch 2/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7999 - loss: 0.4871  
 99/534 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7933 - loss: 0.4437Epoch 2/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7838 - loss: 0.4733
Epoch 2/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8324 - loss: 0.4013
Epoch 3/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8057 - 

/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8705 - loss: 0.3169
Epoch 49/50
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8740 - loss: 0.31
348/534 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8729 - loss: 0.3133Epoch 1/50
493/534 ━━━━━━━━━━━━━━━━━━━━ 0s 922us/step - accuracy: 0.8679 - loss: 0.3166

/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9004 - loss: 0.2318
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8682 - loss: 0.3326
Epoch 5/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8586 - loss: 0.3426
Epoch 4/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8684 - loss: 0.3096
Epoch 18/100
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8655 - loss: 0.3242
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8699 - loss: 0.3018
410/534 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8635 - loss: 0.3319Epoch 15/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8693 - loss: 0.3128
Epoch 20/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8708 - loss: 0.3152
Epoch 12/100
  1/534 ━━━━━━━━━━━━━━━━━━━━ 24s 46ms/step - accuracy: 0.8000 - loss: 0.4083Epoch 1/100


/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8656 - loss: 0.3278
Epoch 6/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8567 - loss: 0.3363
Epoch 5/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8718 - loss: 0.3084
Epoch 19/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8717 - loss: 0.3000
Epoch 16/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8684 - loss: 0.3117
Epoch 21/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8706 - loss: 0.3145
275/534 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7838 - loss: 0.4872Epoch 13/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8669 - loss: 0.3238
Epoch 7/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8243 - loss: 0.4236
Epoch 2/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8723 - loss: 0.3073
508/534 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8651 - loss: 0.3302Epoch 20/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8618 - lo

/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 427us/step - accuracy: 0.7437 - loss: 0.5182
Epoch 2/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 0s 422us/step - accuracy: 0.8216 - loss: 0.4145
Epoch 3/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 0s 419us/step - accuracy: 0.8381 - loss: 0.3861
Epoch 4/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 0s 419us/step - accuracy: 0.8478 - loss: 0.3681
Epoch 5/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 0s 419us/step - accuracy: 0.8526 - loss: 0.3579
Epoch 6/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 0s 420us/step - accuracy: 0.8537 - loss: 0.3522
Epoch 7/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 0s 418us/step - accuracy: 0.8559 - loss: 0.3486
Epoch 8/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 0s 450us/step - accuracy: 0.8580 - loss: 0.3464
Epoch 9/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 0s 426us/step - accuracy: 0.8594 - loss: 0.3449
Epoch 10/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 0s 418us/step - accuracy: 0.8596 - loss: 0.3438
Epoch 11/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 0s 419us/step - accuracy: 0.8594 - loss: 0.3429
Epoch 12/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 